In [2]:
import dotenv
from transformers import LongformerTokenizer, LongformerModel
import torch
from langchain_pinecone import PineconeVectorStore
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.utilities import PythonREPL

# Load environment variables
dotenv.load_dotenv()

/Users/rahulsingh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rahulsingh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# Load Longformer model and tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-large-4096')
model = LongformerModel.from_pretrained('allenai/longformer-large-4096')

In [4]:
# Define custom embedding class with embed_query method
class LongformerEmbedding:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()  # Mean pooling
        return embedding

# Instantiate the custom embedding class
longformer_embedder = LongformerEmbedding(tokenizer, model)

In [5]:
# Connect to Pinecone and initialize with the existing index
index_name = "hubspot-crm-txts"

# Initialize PineconeVectorStore with the custom embedding class
vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=longformer_embedder  # Pass the embedding class here
)

In [31]:
# Set up the Retrieval Chain with LangChain
llm_rag = ChatAnthropic(model='claude-3-5-sonnet-20241022', temperature=0.1)

# Define your custom prompt with context and prefix
rag_template = """
Role: HubSpot code generator that prioritizes context-provided information.

Context: {text}

Generate Python code for HubSpot API operations that:
- Uses HubSpot Client Library when possible, falls back to requests library for associations
- Returns raw JSON responses (no parsing)
- Includes proper error handling
- Uses provided access_token
- Includes necessary imports
- Ends with print(response)

For filtering:
- Use HubSpot Filter objects with correct imports
- Match filter properties to object type

Code requirements:
- No comments/docstrings
- No syntax blocks
- No explanations
- Try-except wrapped
- Raw response only

For associations:
- Use requests.get() with proper endpoints
- Format: api.hubapi.com/crm/v4/objects/<object_type>/<id>/associations/<to_object_type>
- Include Authorization header with Bearer token

Question: {question}
"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["text", "question"])

In [32]:
# Define the Retrieval-Augmented Generation (RAG) chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm_rag,
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    chain_type_kwargs = {
        "prompt": rag_prompt,
        "document_variable_name": "text"
    },   
)

In [33]:
# Query the RAG Chain
query = "what contacts are associated to the deal with id = 28338118010? my access token is pat-na1-59c263ed-698c-4077-b2bc-d1e001750420"
response = rag_chain.invoke(query)

In [34]:
# Setup simple llm to explain the results
llm_explanation = ChatAnthropic(model='claude-3-5-haiku-20241022', temperature=0.5)

explanation_template = """
Explain HubSpot responses in plain language:

Format:
1. Summary line: "Found [X] results for [query type]"
2. Bullet-point details for each result
3. For errors: Explain issue and solution simply

Rules:
- Use everyday language
- No technical terms (API, JSON, etc.)
- Be thorough but simple
- No clarification offers
- No jargon

Style:
- Clear
- Descriptive
- Easy to understand
- Direct

Response: {response}

Use the response to answer the user's query below.

Query: {query}
"""

explanation_prompt = PromptTemplate(template=explanation_template, input_variables=["response", "query"])

In [35]:
# Define the explanation chain
explanation_chain = LLMChain(
    llm=llm_explanation,
    prompt=explanation_prompt
)

In [36]:
# Run the explanation chain
explanation = explanation_chain.run({
    "response": PythonREPL().run(response['result']),
    "query": query
})
print(explanation)

Summary: Found 1 result for deal associations

• One contact is linked to the deal (Deal ID: 28338118010)
• The contact's unique ID is 71750622225
• This is a standard HubSpot system-defined connection

Important Notes:
- The response shows a basic association between a contact and the specified deal
- Only the contact's ID is provided, not additional contact details
- The association type is a standard HubSpot system connection

Recommendation:
To get full contact details, you would need to make a separate request using the contact ID 71750622225.
